## Web Scraping Automatizado: Taylor & Francis

Este notebook implementa un **bot de web scraping automatizado** para descargar artículos científicos en formato BibTeX desde Taylor & Francis.

### Objetivos:
1. **Autenticación automática** vía proxy institucional
2. **Navegación programática** a Taylor & Francis
3. **Búsqueda automatizada** de términos específicos
4. **Descarga masiva** de archivos BibTeX con paginación
5. **Organización** de archivos descargados

### Flujo del Proceso:
```
Login Proxy → Autenticación Google → Selección Facultad → 
Acceso Taylor & Francis → Búsqueda → Manejo de Cookies → 
Loop de Paginación → Descarga BibTeX → Renombrado → Siguiente Página
```

## Paso 1 — Imports y carga de variables de entorno

Este bloque importa las librerías necesarias para manejar datos, controlar el navegador con Selenium y cargar variables desde un archivo .env.

In [ ]:
#imports requeridos para que el proyecto funcione
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import (
    TimeoutException, 
    StaleElementReferenceException,
    NoSuchElementException
)
import time
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

# Obtener y validar variables de entorno
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

# Validación crítica de variables
if not email or not password:
    raise ValueError("ERROR: EMAIL y PASSWORD deben estar definidos en el archivo .env")

print(f"Variables cargadas correctamente")
print(f"Email: {email[:3]}***@{email.split('@')[1] if '@' in email else '***'}")

## Paso 2 — Configuración de ruta y opciones de descarga en Chrome

Este bloque define la carpeta donde se guardarán las descargas y configura las preferencias del navegador Chrome para automatizar ese proceso.

In [ ]:
# Configurar la ruta de descarga
download_path = os.getenv("DOWNLOAD_PATH")

# Validación crítica de ruta de descarga
if not download_path:
    raise ValueError("ERROR: DOWNLOAD_PATH debe estar definido en el archivo .env")

os.makedirs(download_path, exist_ok=True)

chrome_options = webdriver.ChromeOptions()

# Configuración de preferencias de descargas
prefs = {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
chrome_options.add_experimental_option("prefs", prefs)

# Argumentos adicionales para estabilidad
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-download-notification")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

print(f"Opciones de Chrome configuradas")
print(f"Ruta de descarga: {download_path}")

## Paso 3 — Inicialización del navegador y apertura de la página

Este bloque crea una instancia del navegador Chrome con las opciones configuradas previamente y accede a la URL deseada.

In [ ]:
# Iniciar el navegador con protección
driver = None

try:
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    print("Navegador Chrome iniciado correctamente")
    
    # Abrir la URL
    url1 = 'https://login.intelproxy.com/v2/inicio?cuenta=7Ah6RNpGWF22jjyq'
    driver.get(url1)
    print(f"Navegando a: {url1}")
    
except Exception as e:
    print(f"ERROR: Error al iniciar el navegador: {e}")
    if driver:
        driver.quit()
    raise

## Paso 4 — Espera inicial para carga de página

Este bloque realiza una pausa para asegurar que la página se cargue completamente antes de continuar.

In [4]:
time.sleep(5)

## Paso 5 — Esperar y hacer clic en el botón de inicio de sesión

Este bloque espera hasta que el botón de inicio de sesión esté disponible y realiza el clic para continuar con el proceso.

In [5]:
login_button2 = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[2]/a")))
login_button2.click()

## Paso 6 — Espera para transición de página

Este bloque realiza una pausa para permitir la transición a la página de autenticación.

In [6]:
time.sleep(3)

## Paso 7 — Ingresar el correo electrónico

Este bloque espera el campo de correo, introduce el valor obtenido del archivo .env y envía la tecla Enter para continuar.

In [7]:
email_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "identifierId"))
)

email_input.send_keys(email)
email_input.send_keys(Keys.RETURN)

## Paso 8 — Espera para validación de correo

Este bloque realiza una pausa para permitir la validación del correo electrónico.

In [8]:
time.sleep(3)

## Paso 9 — Ingresar la contraseña

Este bloque espera el campo de contraseña, escribe la clave obtenida del archivo .env y envía la tecla Enter para iniciar sesión.

In [9]:
password_input = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.NAME, "Passwd"))
)# Nombre del campo de contraseña
password_input.send_keys(password)
password_input.send_keys(Keys.RETURN)

## Paso 10 — Acceder al apartado "Fac. Ingeniería"

Este bloque localiza y hace clic en el botón que despliega la sección correspondiente a "Fac. Ingeniería".

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Buscar el botón <summary> que contiene "Fac. Ingeniería"
boton = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//summary[contains(., 'Fac. Ingeniería')]"))
)

time.sleep(2)
boton.click()

## Paso 11 — Localizar y acceder a Taylor & Francis

Este bloque expande la sección de "Fac. Ciencias Agroindustriales", localiza el enlace a Taylor & Francis y hace clic para acceder.

In [11]:
# Si el artículo está dentro de un <details> cerrado, expándelo primero
summary = driver.find_element(By.XPATH, "//summary[contains(., 'Fac. Ciencias Agroindustriales')]")
driver.execute_script("arguments[0].scrollIntoView();", summary)
summary.click()
time.sleep(1)

# Ahora busca el enlace
enlaceTaylor = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//article[@id='facingenierataylorfrancisdescubridor']//h2[@class='result-title']/a"))
)
enlaceTaylor.click()

## Paso 12 — Manejo de cookies (primer intento)

Este bloque intenta detectar y aceptar el banner de cookies que puede aparecer al cargar la página de Taylor & Francis.

In [12]:
try:
    # Esperar a que aparezca el botón de aceptar cookies
    boton_cookiesTaylor = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    # Hacer scroll al botón para asegurarse de que sea visible
    driver.execute_script("arguments[0].scrollIntoView(true);", boton_cookiesTaylor)
    time.sleep(0.5)
    # Click con JavaScript para mayor confiabilidad
    driver.execute_script("arguments[0].click();", boton_cookiesTaylor)
    print("✅ Cookies aceptadas correctamente.")
    time.sleep(2)
except Exception as e:
    print(f"⚠ No apareció la ventana de cookies o ya estaba aceptada: {e}")
    # Intentar con selector alternativo
    try:
        boton_alt = driver.find_element(By.CSS_SELECTOR, "button.onetrust-close-btn-handler")
        driver.execute_script("arguments[0].click();", boton_alt)
        print("✅ Cookies aceptadas con selector alternativo.")
    except:
        print("⚠ No se pudo interactuar con cookies, continuando...")

⚠ No apareció la ventana de cookies o ya estaba aceptada: Message: 
Stacktrace:
#0 0x55698c6f4aea <unknown>
#1 0x55698c140cdb <unknown>
#2 0x55698c1936c4 <unknown>
#3 0x55698c193901 <unknown>
#4 0x55698c1e28b4 <unknown>
#5 0x55698c1dfc87 <unknown>
#6 0x55698c185aca <unknown>
#7 0x55698c1867d1 <unknown>
#8 0x55698c6bbab9 <unknown>
#9 0x55698c6bea8c <unknown>
#10 0x55698c6a4d49 <unknown>
#11 0x55698c6bf685 <unknown>
#12 0x55698c68c6c3 <unknown>
#13 0x55698c6e17d8 <unknown>
#14 0x55698c6e19b3 <unknown>
#15 0x55698c6f3a83 <unknown>
#16 0x7f22ff532f54 start_thread
#17 0x7f22ff5b632c __clone3

⚠ No se pudo interactuar con cookies, continuando...


## Paso 13 — Localizar la barra de búsqueda

Este bloque espera a que la barra de búsqueda de Taylor & Francis esté disponible para interactuar con ella.

In [13]:
# Buscar el input de búsqueda por su clase
barra_busquedaTaylor = WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.ID, "searchText-1d85a42e-ad57-4c0d-a477-c847718bcb5d"))
)


## Paso 14 — Ingresar término de búsqueda

Este bloque escribe el término de búsqueda "generative artificial intelligence" en la barra de búsqueda.

In [14]:
# Escribir "generative artificial intelligence"
barra_busquedaTaylor.send_keys('generative artificial intelligence')

## Paso 15 — Ejecutar la búsqueda

Este bloque envía la tecla Enter para ejecutar la búsqueda y cargar los resultados.

In [15]:
barra_busquedaTaylor.send_keys(Keys.RETURN)

## Paso 16 — Manejo de cookies (segundo intento)

Este bloque realiza una segunda verificación para aceptar cookies después de cargar los resultados de búsqueda.

In [16]:
try:
    # Esperar a que aparezca el botón de aceptar cookies (segunda verificación)
    boton_cookiesTaylor = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    # Hacer scroll al botón
    driver.execute_script("arguments[0].scrollIntoView(true);", boton_cookiesTaylor)
    time.sleep(0.5)
    # Click con JavaScript
    driver.execute_script("arguments[0].click();", boton_cookiesTaylor)
    print("✅ Cookies aceptadas correctamente (segunda verificación).")
    time.sleep(2)
except Exception as e:
    print(f"⚠ No apareció la ventana de cookies o ya estaba aceptada: {e}")
    # Intentar con selectores alternativos
    try:
        # Probar con clase CSS
        boton_alt = driver.find_element(By.CSS_SELECTOR, "button.onetrust-close-btn-handler, button#onetrust-accept-btn-handler")
        driver.execute_script("arguments[0].click();", boton_alt)
        print("✅ Cookies aceptadas con selector alternativo.")
    except:
        print("⚠ No se pudo interactuar con cookies, continuando...")

⚠ No apareció la ventana de cookies o ya estaba aceptada: Message: 
Stacktrace:
#0 0x55698c6f4aea <unknown>
#1 0x55698c140cdb <unknown>
#2 0x55698c1936c4 <unknown>
#3 0x55698c193901 <unknown>
#4 0x55698c1e28b4 <unknown>
#5 0x55698c1dfc87 <unknown>
#6 0x55698c185aca <unknown>
#7 0x55698c1867d1 <unknown>
#8 0x55698c6bbab9 <unknown>
#9 0x55698c6bea8c <unknown>
#10 0x55698c6a4d49 <unknown>
#11 0x55698c6bf685 <unknown>
#12 0x55698c68c6c3 <unknown>
#13 0x55698c6e17d8 <unknown>
#14 0x55698c6e19b3 <unknown>
#15 0x55698c6f3a83 <unknown>
#16 0x7f22ff532f54 start_thread
#17 0x7f22ff5b632c __clone3

⚠ No se pudo interactuar con cookies, continuando...


## Paso 17 — Descarga automatizada de resultados en formato .bib

Este bloque implementa la automatización completa para descargar los resultados de búsqueda de Taylor & Francis (Springer) en formato BibTeX, página por página. Incluye:

- **Configuración de carpeta dedicada** (springer)
- **Función de espera de descarga** que verifica archivos completos
- **Selección de todos los artículos** en cada página
- **Descarga en formato BibTeX** con confirmación
- **Manejo de elementos obsoletos** (StaleElementReferenceException)
- **Navegación automática** entre páginas hasta que no haya más resultados
- **Renombrado y organización** de archivos descargados
- **Cierre automático del navegador** al finalizar

In [ ]:
import os, time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# ============================================================================
# CONFIGURACIÓN DE CARPETAS
# ============================================================================

# Configura aquí tu carpeta de descargas base
base_download_path = os.getenv("DOWNLOAD_PATH")

# Crear carpeta springer dentro de descargas
springer_folder = os.path.join(base_download_path, "springer")
if not os.path.exists(springer_folder):
    os.makedirs(springer_folder)
    print(f"Carpeta creada: {springer_folder}")
else:
    print(f"Usando carpeta existente: {springer_folder}")

# Usar la carpeta springer como directorio de trabajo
download_path = springer_folder

def esperar_descarga(download_dir, base_dir, timeout=30):
    """Espera a que aparezca un archivo .bib COMPLETO en la carpeta base de descargas"""
    print(f"Esperando descarga (timeout: {timeout}s)...")
    end_time = time.time() + timeout
    
    while time.time() < end_time:
        try:
            # Buscar archivos que terminen en .bib y no tengan .crdownload en la carpeta BASE
            files = [f for f in os.listdir(base_dir) if f.endswith(".bib")]
            if files:
                full_paths = [os.path.join(base_dir, f) for f in files]
                archivo = max(full_paths, key=os.path.getctime)

                # Verificar que no exista el temporal de Chrome y que tenga contenido
                if not os.path.exists(archivo + ".crdownload") and os.path.getsize(archivo) > 0:
                    print(f"Descarga completada")
                    return archivo
        except Exception as e:
            print(f"ADVERTENCIA: Error al verificar descarga: {e}")
        
        time.sleep(1)
    
    print(f"ERROR: Timeout - No se detectó descarga en {timeout}s")
    return None


# -------------------------------
# Recorrer todas las páginas hasta que no haya "Next"
# -------------------------------
page_num = 1
errores_consecutivos = 0
MAX_ERRORES_CONSECUTIVOS = 3

print(f"\n{'='*60}")
print(f"INICIANDO DESCARGA AUTOMATIZADA DE TAYLOR & FRANCIS")
print(f"{'='*60}\n")

while True:
    print(f"\n{'─'*60}")
    print(f"Procesando página {page_num}")
    print(f"{'─'*60}")

    try:
        # 1. Esperar carga completa
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )

        # 2. Marcar "Select all" con reintentos para StaleElementReferenceException
        for intento in range(3):
            try:
                check_all = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[id^='markall_']"))
                )
                driver.execute_script("arguments[0].click();", check_all)
                time.sleep(1)

                if not check_all.is_selected():
                    driver.execute_script("arguments[0].click();", check_all)

                print("Todos los resultados seleccionados en la página")
                errores_consecutivos = 0  # Resetear contador
                break
            except StaleElementReferenceException:
                print(f"Reintentando marcar 'Select all' (intento {intento + 1}/3)...")
                time.sleep(1)
                if intento == 2:
                    raise
            except TimeoutException:
                print(f"ADVERTENCIA: Timeout al seleccionar checkbox")
                errores_consecutivos += 1
                if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                    raise
                break

        # 3. Abrir modal "Download citations"
        try:
            dl_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.download-citations"))
            )
            driver.execute_script("arguments[0].click();", dl_btn)
            time.sleep(1)
            print("Modal de descarga abierto")
        except TimeoutException:
            print("ADVERTENCIA: No se pudo abrir modal de descarga")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # 4. Seleccionar BibTeX
        try:
            bibtex_radio = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "format-bibtex"))
            )
            driver.execute_script("arguments[0].click();", bibtex_radio)
            time.sleep(0.5)
            print("Formato BibTeX seleccionado")
        except TimeoutException:
            print("ADVERTENCIA: No se pudo seleccionar BibTeX")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # 5. Confirmar descarga
        try:
            dl_confirm = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "btn-download-citations"))
            )
            driver.execute_script("arguments[0].click();", dl_confirm)
            print("Descargando citas en BibTeX...")
        except TimeoutException:
            print("ADVERTENCIA: No se pudo confirmar descarga")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                break
            continue

        # 6. Esperar que se descargue y mover a carpeta springer
        archivo = esperar_descarga(download_path, base_download_path, 30)
        if archivo:
            # Mover el archivo a la carpeta springer con nombre descriptivo
            nuevo_nombre = f"springer_page_{page_num}.bib"
            ruta_nueva = os.path.join(download_path, nuevo_nombre)
            
            # Si el archivo ya existe en springer, eliminarlo primero
            if os.path.exists(ruta_nueva):
                os.remove(ruta_nueva)
            
            os.rename(archivo, ruta_nueva)
            print(f"Archivo guardado como: {nuevo_nombre}")
            errores_consecutivos = 0  # Resetear contador
        else:
            print("ERROR: No se detectó descarga .bib")
            errores_consecutivos += 1
            if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
                print(f"ERROR: Demasiados errores consecutivos. Deteniendo proceso.")
                break

        # 7. Intentar ir a la siguiente página
        try:
            old_element = driver.find_element(By.CSS_SELECTOR, "input[id^='markall_']")

            next_link = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a.nextPage"))
            )
            
            # Verificar que el botón no esté deshabilitado
            if 'disabled' in next_link.get_attribute('class'):
                print("INFO: Botón de siguiente página deshabilitado. Fin del recorrido.")
                break
            
            driver.execute_script("arguments[0].click();", next_link)

            # Esperar a que el elemento viejo desaparezca
            WebDriverWait(driver, 10).until(EC.staleness_of(old_element))

            # Esperar a que aparezca el checkbox de la nueva página
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "input[id^='markall_']"))
            )
            time.sleep(2)

            page_num += 1
            print(f"Avanzando a la página {page_num}...")

        except TimeoutException:
            print("INFO: No hay más páginas, fin del recorrido.")
            break

    except KeyboardInterrupt:
        print("\nADVERTENCIA: Proceso interrumpido por el usuario")
        break
    except Exception as e:
        print(f"ADVERTENCIA: Error en la página {page_num}: {type(e).__name__}: {e}")
        errores_consecutivos += 1
        if errores_consecutivos >= MAX_ERRORES_CONSECUTIVOS:
            print(f"ERROR: Demasiados errores consecutivos. Deteniendo proceso.")
            break
        continue

print(f"\n{'='*60}")
print(f"Descarga completada")
print(f"Total de páginas procesadas: {page_num}")
print(f"Archivos guardados en: {download_path}")
print(f"{'='*60}\n")

# Cerrar el navegador de forma segura
if driver:
    try:
        print("Cerrando navegador...")
        driver.quit()
        print("Navegador cerrado correctamente")
    except Exception as e:
        print(f"ADVERTENCIA: Error al cerrar navegador: {e}")

📁 Usando carpeta existente: /home/yep/Documentos/proyectoAnalisisAlgoritmos/descargas/springer

📄 Procesando página 1...
✅ Todos los resultados seleccionados en la página
⬇️ Descargando citas en BibTeX...
✅ Archivo guardado como: springer_page_1.bib

📄 Procesando página 2...
✅ Todos los resultados seleccionados en la página
⬇️ Descargando citas en BibTeX...
✅ Archivo guardado como: springer_page_2.bib

📄 Procesando página 3...
✅ Todos los resultados seleccionados en la página
⬇️ Descargando citas en BibTeX...
✅ Archivo guardado como: springer_page_3.bib

📄 Procesando página 4...
✅ Todos los resultados seleccionados en la página
⬇️ Descargando citas en BibTeX...
✅ Archivo guardado como: springer_page_4.bib

📄 Procesando página 5...
✅ Todos los resultados seleccionados en la página
⬇️ Descargando citas en BibTeX...
✅ Archivo guardado como: springer_page_5.bib

📄 Procesando página 6...
✅ Todos los resultados seleccionados en la página
⚠️ Error en la página 6: Message: 


🎉 Descarga comple